In [1]:
#########################################
##The model deals with the prediction  ##
##predict required commodities for     ##
##each county each month and the total ##
## required for the next year          ##
#########################################

In [2]:
import pandas as pd
import pickle 
import json

df  = pd.read_csv('data.csv')

#df.head()

In [3]:
#create a timestamp(the format is YYYY-MM-DD)
dateformat  = []

def get_datetime():
    dateformat.clear()
    for timestamp in df['periodid']:

        dates = str(timestamp)[:4]+"-"+str(timestamp)[-2:]

        dateformat.append(dates)

get_datetime()
#print(datetime)

In [4]:
#per county and loop for all
#rename the columns
new_names  = {"Baringo County": 'baringo', "Bomet County": 'bomet', "Bungoma County": 'bungoma', "Busia County": 'busia', "Elgeyo Marakwet County": 'elgeyo',
               "Embu County": 'embu', "Garissa County": 'garissa', "Homa Bay County": 'homabay', "Isiolo County": 'isiolo', "Kajiado County": 'kajiado',
                "Kakamega County": 'kakamega', "Kericho County": 'kericho', "Kiambu County": 'kiambu', "Kilifi County": 'kilifi', "Kirinyaga County": 'kirinyaga',
                "Kisii County": 'kisii', "Kisumu County": 'kisumu', "Kitui County": 'kitui', "Kwale County": 'kwale', "Laikipia County": 'laikipia', "Lamu County": 'lamu',
                "Machakos County": 'machakos', "Makueni County": 'makueni', "Mandera County": 'mandera', "Marsabit County": 'marsabit', "Meru County": 'meru', "Migori County": 'migori',
                "Mombasa County": 'mombasa', "Muranga County": 'muranga', "Nairobi County": 'nairobi', "Nakuru County": 'nakuru', "Nandi County": 'nandi', "Narok County": 'narok',
                "Nyamira County": 'nyamira', "Nyandarua County": 'nyandarua', "Nyeri County": 'nyeri', "Samburu County": 'samburu', "Siaya County": 'siaya', "Taita Taveta County": 'taita', 
                "Tana River County": 'tanariver', "Tharaka Nithi County": 'tharaka', "Trans Nzoia County": 'transzoia', "Turkana County": 'turkana', "Uasin Gishu County": 'uasingishu',
                "Vihiga County": 'vihiga', "Wajir County": 'wajir', "West Pokot County": 'westpokot'}

df.rename(columns=new_names, inplace=True)
months  = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
years = [ periodid for periodid in df['periodid']]

In [5]:
#create a data to be supported during prediction(the format is y)
from math import floor
county_monthly_data = []

def county_year_consumption(county):
   
    """This county_year_consumption function gives out the total commodities dispatched a certain complete year
       for a particular county
    """
    #run above function to get the year periods in place
    get_datetime()
    county_monthly_data.clear()
    try:
      for periodid in years:
         val = df.loc[df['periodid']==int(periodid)].sum()
         
         commodity_total = val[county]
         if commodity_total == 0:
            county_monthly_data.append(0)
         else:
            county_monthly_data.append(floor(commodity_total))
         #print(f"{county} has {commodity_total} kits in {periodid}")
      
       
    except Exception as e:
       
       if e.__class__.__name__ == 'KeyError':
          print("Check if the county name is accurate.Or ask the lead developer for solution.")



In [6]:
import numpy as np
#predict using the prophet
from prophet import Prophet
from prophet.diagnostics import performance_metrics, cross_validation
import datetime

predicted_data = []

def predict(county):
    #create a dataframe
    #Have the data in a dictionary 
    #the first key will be ds while the other y
    # Create a dataframe with a datetime column and a corresponding numeric column
    county_year_consumption(county)
    #convert the list of datetime to a pandas datetime object
    datetime_index  = pd.to_datetime(dateformat, format='%Y-%m')
    #Extract the end month of the month for each month
    end_of_month = datetime_index + pd.offsets.MonthBegin(1)
    #print(end_of_month)
    dataframe  = {'ds':end_of_month , 'y': county_monthly_data}

    new_df = pd.DataFrame(dataframe)

    #new_df.head()
    # Instantiate a Prophet object and fit the model to the data
    model  = Prophet()

    model.fit(new_df)

    # Create a dataframe with dates to make a prediction for
    future  = model.make_future_dataframe(periods=11, freq='MS')

    # Use the model to make a prediction for the next 12 months
    forecast  = model.predict(future)

    #print the predicted values for the next 12 months
    forecasted = forecast[['ds', 'yhat']]

    ds_values = forecasted.tail(12)['ds'].values
    yhat_values = forecasted.tail(12)['yhat'].values

    for date_time,predicted_value,month in zip(ds_values,yhat_values,months):
        actual_datetime = pd.to_datetime(date_time).strftime('%Y-%m-%d')
        
        predicted_data.append(f"Prediction for {county} in {month} on {actual_datetime} is {floor(predicted_value)}")
    
    
    #model.plot(forecast)


Importing plotly failed. Interactive plots will not work.


In [7]:
#A fucntion that finally executes prediction for all counties
def execute_prediction():
    for k,v in new_names.items():

        predict(v)
    with open('county_prediction.json', 'w+') as f:

        f.write(json.dumps(predicted_data)) 

execute_prediction()

16:49:41 - cmdstanpy - INFO - Chain [1] start processing
16:49:41 - cmdstanpy - INFO - Chain [1] done processing
16:49:41 - cmdstanpy - INFO - Chain [1] start processing


baringo


16:49:42 - cmdstanpy - INFO - Chain [1] done processing
16:49:42 - cmdstanpy - INFO - Chain [1] start processing


bomet


16:49:42 - cmdstanpy - INFO - Chain [1] done processing
16:49:42 - cmdstanpy - INFO - Chain [1] start processing


...

vihiga


16:50:04 - cmdstanpy - INFO - Chain [1] done processing
16:50:04 - cmdstanpy - INFO - Chain [1] start processing


wajir


16:50:04 - cmdstanpy - INFO - Chain [1] done processing


westpokot
